# Apprentissage non supervisé des paramètres du HMM
* Ne fonctionne pas mais dans l'idée le code est bon
* Tous les calculs sont implementés et il y a des commentaire


In [1]:
import nltk
from numpy import array, ones, zeros, multiply
import numpy as np
import sys
from pprint import pprint
import operator

epsilon=1e-1

class HMM:
        def __init__(self, state_list, observation_list,
                 transition_proba = None,
                 observation_proba = None,
                 initial_state_proba = None, smoothing_obs = 0.01):
            print "HMM creating with: "
            self.N = len(state_list)       # number of states
            self.M = len(observation_list) # number of possible emissions
            print str(self.N)+" states"
            print str(self.M)+" observations"
            self.omega_Y = state_list
            self.omega_X = observation_list
            if transition_proba is None:
                self.transition_proba = zeros( (self.N, self.N), float) 
            else:
                self.transition_proba=transition_proba
            if observation_proba is None:
                self.observation_proba = zeros( (self.M, self.N), float) 
            else:
                self.observation_proba=observation_proba
            if initial_state_proba is None:
                self.initial_state_proba = zeros( (self.N,), float ) 
            else:
                self.initial_state_proba=initial_state_proba
            self.make_indexes() # build indexes, i.e the mapping between token and int
            self.smoothing_obs = smoothing_obs 
            
        def make_indexes(self):
            """Creates the reverse table that maps states/observations names
            to their index in the probabilities array"""
            self.Y_index = {}
            for i in range(self.N):
                self.Y_index[self.omega_Y[i]] = i
            self.X_index = {}
            for i in range(self.M):
                self.X_index[self.omega_X[i]] = i
      
        def get_observationIndices( self, observations ):
            """return observation indices, i.e 
            return [self.O_index[o] for o in observations]
            and deals with OOVs
            """
            indices = zeros( len(observations), int )
            k = 0
            for o in observations:
                if o in self.X_index:
                    indices[k] = self.X_index[o]
                else:
                    raise("Erreur")
                k += 1
            return indices

    
        def data2indices(self, sent): 
            """From one tagged sentence of the brown corpus: 
            - extract the words and tags 
            - returns two list of indices, one for each
            -> (wordids, tagids)
            """
            wordids = list()
            tagids  = list()
            for couple in sent:
                wrd = couple[0]
                tag = couple[1]
                if wrd in self.X_index:
                    wordids.append(self.X_index[wrd])
                else:
                    raise("Erreur data2indices")
                tagids.append(self.Y_index[tag])
            return wordids,tagids
            
        def observation_estimation(self, pair_counts):
            """ Build the observation distribution: 
                observation_proba is the observation probablility matrix
                    [b_ki],  b_ki = Pr(X_t=v_k|Y_t=q_i)"""
            # fill with counts
            print("pair_counts" , len(set(pair_counts.keys())))
            for pair in pair_counts:
                wrd=pair[0]
                tag=pair[1]
                cpt=pair_counts[pair]
                k = 0 # for <unk>
                if wrd in self.X_index: 
                    k=self.X_index[wrd]
                i=self.Y_index[tag]
                self.observation_proba[k,i]=cpt
            # normalize
            self.observation_proba=self.observation_proba+self.smoothing_obs
            self.observation_proba=self.observation_proba/self.observation_proba.sum(axis=0).reshape(1,self.N)
            
        
        def transition_estimation(self, trans_counts):
            """ Build the transition distribution: 
                transition_proba is the transition matrix with : 
                [a_ij] a[i,j] = Pr(Y_(t+1)=q_i|Y_t=q_j)
            """
            # fill with counts
            for pair in trans_counts:
                i=self.Y_index[pair[1]]
                j=self.Y_index[pair[0]]
                self.transition_proba[j,i]=trans_counts[pair]
            # normalize
            self.transition_proba=self.transition_proba/self.transition_proba.sum(axis=0).reshape(1,self.N)
        
        def init_estimation(self, init_counts):
            """Build the init. distribution"""
            # fill with counts
            for tag in init_counts:
                i=self.Y_index[tag]
                self.initial_state_proba[i]=init_counts[tag]
            # normalize
            self.initial_state_proba=self.initial_state_proba/sum(self.initial_state_proba)
             
        
        def supervised_training(self, pair_counts, trans_counts,init_counts):
            """ Train the HMM's parameters. This function wraps everything"""
            self.observation_estimation(pair_counts)
            self.transition_estimation(trans_counts)
            self.init_estimation(init_counts)
        
        def viverbit(self,mots):
            alpha = np.zeros((self.N,len(mots)))
            xi = np.zeros((self.N,len(mots)))
            #init
            #print(self.observation_estimation)
            i = 0
            if mots[0] in self.X_index:
                i = self.X_index[mots[0]]
            alpha[:,0] = self.initial_state_proba*self.observation_proba[i]
            for i in range(1,len(mots)):
                for j in range(self.N):
                    #self.observation_proba : mot puis j : type
                    indi = 0
                    if mots[i] in self.X_index:
                        indi = self.X_index[mots[i]]
                    liste = [alpha[k,i-1] * self.transition_proba[k,j]* self.observation_proba[indi,j] for k in range(self.N)]
                    alpha[j,i] = np.max(liste)
                    xi[j,i] = np.argmax(liste)
            tags = []
            debut = len(xi)
            starting = np.argmax(alpha[:,len(xi[0])-1])
            tags.append(starting)
            count = len(xi[0])
            '''print(alpha)
            print(xi)'''
            while len(tags) != len(mots):
                count -= 1
                new_index = xi[starting,count]
                tags.append(new_index)
                starting = new_index
            tags = tags[::-1]
            to_return  = []
            count = 0
            for i in tags:
                to_return.append((mots[count],self.Y_index.keys()[self.Y_index.values().index(i)]))
                count +=1
            return to_return
                 
        def evaluate(self,test_data):
            errors = 0
            total = 0
            erreur_false = 0
            total_false = 0
            erreur_2 = 0
            
            correction = 0
            correction_totale = 0
            for i in range(len(test_data)):
                res = self.viverbit(map(operator.itemgetter(0), test_data[i]))
                if sum([a!=b for a,b in test_data[i]])>0:
                    total_false +=1 
                    erreur_false += sum([a[1]!=b[1] for a,b in zip(res,test_data[i])])
                for a,b in zip(res,test_data[i]):
                    if(b[0] != b[1] and b[1]==a[1]):
                        correction +=1
                    correction_totale +=1
                erreur_2 += sum([a!=b for a,b in test_data[i]])
                errors += sum([a[1]!=b[1] for a,b in zip(res,test_data[i])])
                total += len(res)
            print("Percentage of errors : " ,  (errors/float(total))*100.0)
            print("Pourcentage de correction  : ", (erreur_false/float(total_false)))
            print("Taux d erreur brut " , ((erreur_2/float(total))*100.0))
            print(total)
            print("taux correction calcul  2 " , correction, ((correction/float(correction_totale))*100.0),correction_totale)

# Compter les mots et les tags

In [2]:
def make_counts(corpus):
    """ 
    Build different count tables to train a HMM. Each count table is a dictionnary. 
    Returns: 
    * c_words: word counts
    * c_tags: tag counts
    * c_pairs: count of pairs (word,tag)
    * c_transitions: count of tag bigram 
    * c_inits: count of tag found in the first position
    """
    c_words = dict()
    c_tags = dict()
    c_pairs= dict()
    c_transitions = dict()
    c_inits = dict()
    for sent in corpus:
        # we use i because of the transition counts
        for i in range(len(sent)):
            couple=sent[i]
            wrd = couple[0]
            tag = couple[1]
            # word counts
            if wrd in c_words:
                c_words[wrd]=c_words[wrd]+1
            else:
                c_words[wrd]=1
            # tag counts
            if tag in c_tags:
                c_tags[tag]=c_tags[tag]+1
            else:
                c_tags[tag]=1
            # observation counts
            if couple in c_pairs:
                c_pairs[couple]=c_pairs[couple]+1
            else:
                c_pairs[couple]=1
            # i >  0 -> transition counts
            if i > 0:
                trans = (sent[i-1][1],tag)
                if trans in c_transitions:
                    c_transitions[trans]=c_transitions[trans]+1
                else:
                    c_transitions[trans]=1
            # i == 0 -> counts for initial states
            else:
                if tag in c_inits:
                    c_inits[tag]=c_inits[tag]+1
                else:
                    c_inits[tag]=1
                    
    return c_words,c_tags,c_pairs, c_transitions, c_inits


# Création du vocabulaire (filtrage selon le nombre d'occurence)

In [3]:
def make_vocab(c_words, threshold):
    """ 
    return a vocabulary by thresholding word counts. 
    inputs: 
    * c_words : a dictionnary that maps word to its counts
    * threshold: count must be >= to the threshold to be included
    
    returns: 
    * a word list
    """
    voc = list()
    for w in c_words:
        if c_words[w] >= threshold:
            voc.append(w)
    return voc


# les données


In [4]:
import cPickle

with open("typos-data/test10.pkl", "rb") as input_file:
    test = cPickle.load(input_file)
    
with open("typos-data/train10.pkl", "rb") as input_file:
    train = cPickle.load(input_file)
print "Nombre de phrases de train = "+str(len(train))
print "Nombre de phrases de test  = "+str(len(test))

Nombre de phrases de train = 29057
Nombre de phrases de test  = 1501


In [5]:
cwords,ctags,cpairs,ctrans,cinits = make_counts(train)
print "Nombre de mots  : "+str(len(cwords))
print "Nombre de tags  : "+str(len(ctags))
print "Nombre de paires: "+str(len(cpairs))
print "Nombre de trans : "+str(len(ctrans))+ " / "+ str(12*12)
print "Nombre de init. : "+str(len(cinits))
vocab = make_vocab(cwords,10)
print "Vocabulaire :"+str(len(vocab))

Nombre de mots  : 26
Nombre de tags  : 26
Nombre de paires: 127
Nombre de trans : 403 / 144
Nombre de init. : 25
Vocabulaire :26


# Création du HMM

In [6]:
hmm = HMM(state_list=ctags.keys(), observation_list=vocab,
                 transition_proba = None,
                 observation_proba = None,
                 initial_state_proba = None,
                 smoothing_obs = 1e-2)
hmm.supervised_training(cpairs,ctrans,cinits)

HMM creating with: 
26 states
26 observations
('pair_counts', 127)


# Baum-Welch

basé sur http://vision.gel.ulaval.ca/~parizeau/Publications/P971225.pdf

In [8]:
import numpy as np

class unsupervised_hmm:
    def __init__(self,train,test,state_list,observation_list):
        #calcul du nombre de labels possibles
        self.N = len(state_list)       # number of states
        self.M = len(observation_list) # number of possible emissions
        #A : proba de transition
        #B : proba d'observation
        #initialsiation alétaoire
        pi,A,B = self.random_val(self.N,self.M)
        
        #critère de convergence
        has_converged = False
        nb_iters = 10
        
        #cree l'index des observations
        self.make_indexes(observation_list)
        
        #supprime les mots de taille <= 1
        train_2 = []
        for t in train:
            if len(t)>1:
                train_2.append(t)
        train = train_2
        
        
        while not has_converged and nb_iters>0:          
            #nouveaux paramètres du hmm
            tmpP,tmpA,tmpB = self.EM(pi,A,B,train,self.N,self.M)
            pi = tmpP
            A = tmpA
            B = tmpB
            nb_iters -= 25
        
        #algorithme terminé, appelle le hmm pour tester avec les paramètres calculés par l'algorithme
        hmm = HMM(state_list=ctags.keys(), observation_list=vocab,
                 transition_proba = A,
                 observation_proba = B,
                 initial_state_proba = pi)
        print(hmm.viverbit(map(operator.itemgetter(0), test[9])))
        
    
    def make_indexes(self,omega_Y):
        #associe a chaque label un unique identifiant
        self.Y_index = {}
        for i in range(self.N):
            self.Y_index[omega_Y[i]] = i
            
    def EM(self,pi,X,Y,obsevrations,N,M):
        #e phase : stimation 
        
        
        #pour chaque donnée d'apprentissage calcule :
        #gamma : proba d'être dans l'état i à l'instant t
        #xi : proba d'être dans l'état i  à l'instant t et dans l'état j à l'instant t+1 pour une séquence d'observations
        #alpha : proba d'observé une sequence d'observation de taille t<=T lorsqu'on se trouve dans l'etat q
        #beta :proba d'être dans l'état q en ayant observé une sequence d'observation de taille t (t<T)
        my_gamma = []
        my_xi = []
        for i in range(len(obsevrations)):
            obs = np.array(obsevrations[i])
            alpha = self.forward(obs,N,M,pi,X,Y)
            #calcul de beta
            beta = self.backward(obs,N,M,pi,X,Y)
            #calcul proba
            gamma, xi = self.exceptation(obs,N,M,pi,X,Y,alpha,beta)
            my_gamma.append(gamma)
            my_xi.append(xi)
        
        #nouveaux paramètres
        new_pi = np.zeros(N)
        new_A = np.zeros((N, N))
        new_B = np.zeros((N, M))
        
        #PHASE MAXIMISATION
        #reestimation des paramètres
        
        #calcul proba initial
        for k in range(len(my_gamma)):
            gamma = my_gamma[k]
            new_pi += gamma[:, 0]
        new_pi = new_pi/len(my_gamma)
        
        
        #transition
        #mise à jour des probas d'observation et de transition
        my_xi = np.array(my_xi)
        for i in range(N):
            for j in range(N):
                tmp = 0
                test = 0
                #for k in range(len(my_xi)):
                #    tmp += sum(my_xi[k][:-1, i, j])
                for k in range(len(my_xi)):
                    for t in range(len(my_xi[k])):
                        tmp += my_xi[k][t][i][j]
                        test += my_gamma[k][i][t]
                new_A[i,j] = tmp/test
        for i in range(N):
            dessous = 0
            for k in range(len(my_gamma)):
                for t in range(len(my_xi[k])):
                    dessous += my_gamma[k][i][t]
            for j in range(M):
                test = 0
                for k in range(len(obsevrations)):
                    obs = np.array(obsevrations[k])
                    for tk in range(obs.shape[0]):
                        if self.Y_index[obsevrations[k][tk][0]] == j:
                            if k < len(my_gamma):
                                test += np.sum(my_gamma[k][tk])
                new_B[i,j] = test / dessous
        return (new_pi,new_A,new_B)
    
    
    #calcule de forward
    def forward(self,observations, N , M , pi , A , B):
        #A : proba transition
        #B : proba classe
        T = observations.shape[0]
        alphas = np.zeros((N,T))
        i = 0
        if observations[0][0] in self.Y_index:
            i = self.Y_index[observations[0][0]]
        alphas[:,0] = pi*B[i]
        for t in range(1,T):
            for j in range(N):
                accu = sum([alphas[k,t-1]*A[k,j] for k in range(N)])
                alphas[j,t] = accu * B[self.Y_index[observations[t][0]],j]
        return alphas
    
    #calcule du backward
    def backward(self,observations, N , M , pi , A ,B):
        #A : proba transition
        #B : proba classe
        T = observations.shape[0]
        betas = np.zeros((N, T))
        betas[:,T-1] = 1 
        """ Backward Updates """
        for t in range(T-2 , -1, -1):
            for i in range(N):
                accu = sum([B[self.Y_index[observations[t+1][0]],j]*betas[j,t+1]*A[j,i]for j in range(N)])
                betas[i,t] = accu 
        return betas
    
    
    #calcul de gamma et xi
    def exceptation(self,obs,N,M,pi,A,B,alpha,beta):
        #joint event xi
        T = obs.shape[0]
        #xi : proba d etre a instant t dans etat si  et dans etat j a t+1
        xi = np.zeros((T,N,N))
        gamma  = np.zeros((N,T))
        for t in range(T-1):
            for i in range(N):
                for j in range(N):
                    above = (alpha[i,t] * A[i,j] * B[self.Y_index[obs[t+1][0]],j] * beta[j,t+1])
                    xi[t,i,j] = above
        for t in range(T):
            for i  in range(N):
                gamma[i,t] = sum([xi[t,i,j]for j in range(N)])
        return gamma,xi
    
    #initialisation aléatoire des paramètres
    def random_val(self,N,M):
        X = np.zeros((N, N))
        Y = np.zeros((N, M))
        pi = np.zeros(N)
        #initialisation aleatoire
        pi = np.random.random(N)
        #normalisation
        pi = pi / pi.sum()
        for i in range(N):
            tmp = np.random.random(N)
            X[:, i] = tmp / tmp.sum()
            tmp = np.random.random(M)
            Y[:, i] = tmp / tmp.sum()
        return (pi,X,Y)

print("Test fait sur 10 itérations et 5000 données d'apprentissage")
uhmm = unsupervised_hmm(train[:5000],test,ctags.keys(),vocab)

HMM creating with: 
26 states
26 observations
[('r', 'f'), ('e', 's'), ('e', 'f'), ('l', 's'), ('j', 'f'), ('h', 's'), ('g', 'f'), ('s', 'e')]


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:157: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
